In [1]:
import numpy as np
import pandas as pd

import time

import matplotlib.pyplot as plt
import os
import re
import seaborn as sns


In [2]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /home/cdsw/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [3]:
from textblob import TextBlob

In [4]:
from scipy.stats import moment
from scipy.stats import describe


In [5]:
def basic_stats_error(datalist):
    
    if len(datalist)>1:
        result_describe = list(describe(datalist)[2:])
        result_percentile = result_describe + list(np.percentile(datalist,[10,25,50,75,90]))
        array = np.array(datalist)
        result_percentile = result_percentile+[len(array),len(array[np.where(array>0)[0]])/len(array)*100,
                                               len(array[np.where(array<0)[0]])/len(array)*100]
    else:
        result_percentile = []

    return result_percentile

In [6]:
import seaborn as sns

In [7]:
sns.set(style="whitegrid")

In [8]:
from importlib import reload

from nlp_pandas_functions import ngram_analysis
from nlp_pandas_functions import noun_sentences

reload(ngram_analysis)
reload(noun_sentences)
dir(ngram_analysis)

2022-05-09 23:55:40,031 loading file /home/cdsw/.flair/models/sentiment-en-mix-distillbert_4.pt


INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded
module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'bigrams_join',
 'count_n_grams',
 'defaultdict',
 'df_get_n_grams_count',
 'df_lambda_ngram',
 'generate_n_grams',
 'get_imp',
 'get_n_grams_count',
 'get_n_grams_probability',
 'kg',
 'ngrams',
 'np',
 'pd',
 'text']

In [9]:
column = 'clean_publicremarks'

In [10]:
import glob
filtered_files = glob.glob("comments/raw_comments/*_filtered.fea")

In [11]:
filtered_files

['comments/raw_comments/agent_comments_AL_filtered.fea',
 'comments/raw_comments/agent_comments_AR_filtered.fea',
 'comments/raw_comments/agent_comments_AZ_filtered.fea',
 'comments/raw_comments/agent_comments_CA_filtered.fea',
 'comments/raw_comments/agent_comments_CO_filtered.fea',
 'comments/raw_comments/agent_comments_CT_filtered.fea',
 'comments/raw_comments/agent_comments_DC_filtered.fea',
 'comments/raw_comments/agent_comments_DE_filtered.fea',
 'comments/raw_comments/agent_comments_FL_filtered.fea',
 'comments/raw_comments/agent_comments_GA_filtered.fea',
 'comments/raw_comments/agent_comments_HI_filtered.fea',
 'comments/raw_comments/agent_comments_IA_filtered.fea',
 'comments/raw_comments/agent_comments_ID_filtered.fea',
 'comments/raw_comments/agent_comments_IL_filtered.fea',
 'comments/raw_comments/agent_comments_IN_filtered.fea',
 'comments/raw_comments/agent_comments_KS_filtered.fea',
 'comments/raw_comments/agent_comments_KY_filtered.fea',
 'comments/raw_comments/agent_c

In [11]:
#from nlp_pandas_functions import ngram_analysis
from nlp_pandas_functions import noun_sentences

#reload(ngram_analysis)
reload(noun_sentences)
#dir(ngram_analysis)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


<module 'nlp_pandas_functions.noun_sentences' from '/home/cdsw/nlp_pandas_functions/noun_sentences.py'>

In [21]:
for file in filtered_files:
    
    print(file)
    df_sales = pd.read_feather(file)
    print(len(df_sales))
    tic = time.time()
    
    #df_sales = df_sales.head(10)
    
    result = noun_sentences.df_column_noun_sentences_no_embedding(df_sales, column, 'avmerror', 'ListingPrice', 
                                                 'Transprice', 'avmValue')
    noun_sentence_df = pd.DataFrame([result[0], result[1], result[2], result[3], result[4], result[5]])
    noun_sentence_df_T = noun_sentence_df.T
    noun_sentence_df_T = noun_sentence_df_T.rename(columns={0: 'count', 1: 'AVM_Error', 2: 'AVM_Error_List', 
                                                        3: 'Listing_Error', 4: 'Listing_Error_List',
                                                        5: 'Listingid_List'})
    
    noun_sentence_df_T['AVM_Error_Average'] = noun_sentence_df_T['AVM_Error'] / noun_sentence_df_T['count']
    noun_sentence_df_T['Listing_Error_Average'] = noun_sentence_df_T['Listing_Error'] / noun_sentence_df_T['count']
    
    noun_sentence_df_T = noun_sentence_df_T.sort_values(by ='AVM_Error_Average', ascending=False)
    noun_sentence_df_T_filtered = noun_sentence_df_T[noun_sentence_df_T['count'] > 100]
    noun_sentence_df_T_ri = noun_sentence_df_T.reset_index()
    
    noun_sentence_df_T_ri = noun_sentence_df_T_ri.rename(columns={"index": "noun_sentence"})
    
    noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri[noun_sentence_df_T_ri['count'].notna()]
    
    noun_sentence_df_T_ri_filtered['AVM_Error_Indicators'] = noun_sentence_df_T_ri_filtered['AVM_Error_List'].apply(lambda x: basic_stats_error(x))
    noun_sentence_df_T_ri_filtered['Listing_Error_Indicators'] = noun_sentence_df_T_ri_filtered['Listing_Error_List'].apply(lambda x: basic_stats_error(x))
    
    #noun_sentence_df_T_ri_filtered_2 = noun_sentence_df_T_ri_filtered[noun_sentence_df_T_ri_filtered['count'] > 100]
    
    noun_file = file.split('.')[0] + '_noun_sentences.fea'
    noun_sentence_df_T_ri_filtered.to_feather(noun_file)


comments/raw_comments/agent_comments_AL_filtered.fea
219915
0 219915
1000 219915
2000 219915
3000 219915
4000 219915
5000 219915
6000 219915
7000 219915
8000 219915
9000 219915
10000 219915
11000 219915
12000 219915
13000 219915
14000 219915
15000 219915
16000 219915
17000 219915
18000 219915
19000 219915
20000 219915
21000 219915
22000 219915
23000 219915
24000 219915
25000 219915
26000 219915
27000 219915
28000 219915
29000 219915
30000 219915
31000 219915
32000 219915
33000 219915
34000 219915
35000 219915
36000 219915
37000 219915
38000 219915
39000 219915
40000 219915
41000 219915
42000 219915
43000 219915
44000 219915
45000 219915
46000 219915
47000 219915
48000 219915
49000 219915
50000 219915
51000 219915
52000 219915
53000 219915
54000 219915
55000 219915
56000 219915
57000 219915
58000 219915
59000 219915
60000 219915
61000 219915
62000 219915
63000 219915
64000 219915
65000 219915
66000 219915
67000 219915
68000 219915
69000 219915
70000 219915
71000 219915
72000 219915
7300

276000 405421
277000 405421
278000 405421
279000 405421
280000 405421
281000 405421
282000 405421
283000 405421
284000 405421
285000 405421
286000 405421
287000 405421
288000 405421
289000 405421
290000 405421
291000 405421
292000 405421
293000 405421
294000 405421
295000 405421
296000 405421
297000 405421
298000 405421
299000 405421
300000 405421
301000 405421
302000 405421
303000 405421
304000 405421
305000 405421
306000 405421
307000 405421
308000 405421
309000 405421
310000 405421
311000 405421
312000 405421
313000 405421
314000 405421
315000 405421
316000 405421
317000 405421
318000 405421
319000 405421
320000 405421
321000 405421
322000 405421
323000 405421
324000 405421
325000 405421
326000 405421
327000 405421
328000 405421
329000 405421
330000 405421
331000 405421
332000 405421
333000 405421
334000 405421
335000 405421
336000 405421
337000 405421
338000 405421
339000 405421
340000 405421
341000 405421
342000 405421
343000 405421
344000 405421
345000 405421
346000 405421
347000

429000 2103405
430000 2103405
431000 2103405
432000 2103405
433000 2103405
434000 2103405
435000 2103405
436000 2103405
437000 2103405
438000 2103405
439000 2103405
440000 2103405
441000 2103405
442000 2103405
443000 2103405
444000 2103405
445000 2103405
446000 2103405
447000 2103405
448000 2103405
449000 2103405
450000 2103405
451000 2103405
452000 2103405
453000 2103405
454000 2103405
455000 2103405
456000 2103405
457000 2103405
458000 2103405
459000 2103405
460000 2103405
461000 2103405
462000 2103405
463000 2103405
464000 2103405
465000 2103405
466000 2103405
467000 2103405
468000 2103405
469000 2103405
470000 2103405
471000 2103405
472000 2103405
473000 2103405
474000 2103405
475000 2103405
476000 2103405
477000 2103405
478000 2103405
479000 2103405
480000 2103405
481000 2103405
482000 2103405
483000 2103405
484000 2103405
485000 2103405
486000 2103405
487000 2103405
488000 2103405
489000 2103405
490000 2103405
491000 2103405
492000 2103405
493000 2103405
494000 2103405
495000 210

976000 2103405
977000 2103405
978000 2103405
979000 2103405
980000 2103405
981000 2103405
982000 2103405
983000 2103405
984000 2103405
985000 2103405
986000 2103405
987000 2103405
988000 2103405
989000 2103405
990000 2103405
991000 2103405
992000 2103405
993000 2103405
994000 2103405
995000 2103405
996000 2103405
997000 2103405
998000 2103405
999000 2103405
1000000 2103405
1001000 2103405
1002000 2103405
1003000 2103405
1004000 2103405
1005000 2103405
1006000 2103405
1007000 2103405
1008000 2103405
1009000 2103405
1010000 2103405
1011000 2103405
1012000 2103405
1013000 2103405
1014000 2103405
1015000 2103405
1016000 2103405
1017000 2103405
1018000 2103405
1019000 2103405
1020000 2103405
1021000 2103405
1022000 2103405
1023000 2103405
1024000 2103405
1025000 2103405
1026000 2103405
1027000 2103405
1028000 2103405
1029000 2103405
1030000 2103405
1031000 2103405
1032000 2103405
1033000 2103405
1034000 2103405
1035000 2103405
1036000 2103405
1037000 2103405
1038000 2103405
1039000 2103405


1490000 2103405
1491000 2103405
1492000 2103405
1493000 2103405
1494000 2103405
1495000 2103405
1496000 2103405
1497000 2103405
1498000 2103405
1499000 2103405
1500000 2103405
1501000 2103405
1502000 2103405
1503000 2103405
1504000 2103405
1505000 2103405
1506000 2103405
1507000 2103405
1508000 2103405
1509000 2103405
1510000 2103405
1511000 2103405
1512000 2103405
1513000 2103405
1514000 2103405
1515000 2103405
1516000 2103405
1517000 2103405
1518000 2103405
1519000 2103405
1520000 2103405
1521000 2103405
1522000 2103405
1523000 2103405
1524000 2103405
1525000 2103405
1526000 2103405
1527000 2103405
1528000 2103405
1529000 2103405
1530000 2103405
1531000 2103405
1532000 2103405
1533000 2103405
1534000 2103405
1535000 2103405
1536000 2103405
1537000 2103405
1538000 2103405
1539000 2103405
1540000 2103405
1541000 2103405
1542000 2103405
1543000 2103405
1544000 2103405
1545000 2103405
1546000 2103405
1547000 2103405
1548000 2103405
1549000 2103405
1550000 2103405
1551000 2103405
1552000 

...

1726000 2278007
1727000 2278007
1728000 2278007
1729000 2278007
1730000 2278007
1731000 2278007
1732000 2278007
1733000 2278007
1734000 2278007
1735000 2278007
1736000 2278007
1737000 2278007
1738000 2278007
1739000 2278007
1740000 2278007
1741000 2278007
1742000 2278007
1743000 2278007
1744000 2278007
1745000 2278007
1746000 2278007
1747000 2278007
1748000 2278007
1749000 2278007
1750000 2278007
1751000 2278007
1752000 2278007
1753000 2278007
1754000 2278007
1755000 2278007
1756000 2278007
1757000 2278007
1758000 2278007
1759000 2278007
1760000 2278007
1761000 2278007
1762000 2278007
1763000 2278007
1764000 2278007
1765000 2278007
1766000 2278007
1767000 2278007
1768000 2278007
1769000 2278007
1770000 2278007
1771000 2278007
1772000 2278007
1773000 2278007
1774000 2278007
1775000 2278007
1776000 2278007
1777000 2278007
1778000 2278007
1779000 2278007
1780000 2278007
1781000 2278007
1782000 2278007
1783000 2278007
1784000 2278007
1785000 2278007
1786000 2278007
1787000 2278007
1788000 

2239000 2278007
2240000 2278007
2241000 2278007
2242000 2278007
2243000 2278007
2244000 2278007
2245000 2278007
2246000 2278007
2247000 2278007
2248000 2278007
2249000 2278007
2250000 2278007
2251000 2278007
2252000 2278007
2253000 2278007
2254000 2278007
2255000 2278007
2256000 2278007
2257000 2278007
2258000 2278007
2259000 2278007
2260000 2278007
2261000 2278007
2262000 2278007
2263000 2278007
2264000 2278007
2265000 2278007
2266000 2278007
2267000 2278007
2268000 2278007
2269000 2278007
2270000 2278007
2271000 2278007
2272000 2278007
2273000 2278007
2274000 2278007
2275000 2278007
2276000 2278007
2277000 2278007
2278000 2278007
comments/raw_comments/agent_comments_UT_filtered.fea
307190
0 307190
1000 307190
2000 307190
3000 307190
4000 307190
5000 307190
6000 307190
7000 307190
8000 307190
9000 307190
10000 307190
11000 307190
12000 307190
13000 307190
14000 307190
15000 307190
16000 307190
17000 307190
18000 307190
19000 307190
20000 307190
21000 307190
22000 307190
23000 307190
2

240000 353734
241000 353734
242000 353734
243000 353734
244000 353734
245000 353734
246000 353734
247000 353734
248000 353734
249000 353734
250000 353734
251000 353734
252000 353734
253000 353734
254000 353734
255000 353734
256000 353734
257000 353734
258000 353734
259000 353734
260000 353734
261000 353734
262000 353734
263000 353734
264000 353734
265000 353734
266000 353734
267000 353734
268000 353734
269000 353734
270000 353734
271000 353734
272000 353734
273000 353734
274000 353734
275000 353734
276000 353734
277000 353734
278000 353734
279000 353734
280000 353734
281000 353734
282000 353734
283000 353734
284000 353734
285000 353734
286000 353734
287000 353734
288000 353734
289000 353734
290000 353734
291000 353734
292000 353734
293000 353734
294000 353734
295000 353734
296000 353734
297000 353734
298000 353734
299000 353734
300000 353734
301000 353734
302000 353734
303000 353734
304000 353734
305000 353734
306000 353734
307000 353734
308000 353734
309000 353734
310000 353734
311000

440000 904220
441000 904220
442000 904220
443000 904220
444000 904220
445000 904220
446000 904220
447000 904220
448000 904220
449000 904220
450000 904220
451000 904220
452000 904220
453000 904220
454000 904220
455000 904220
456000 904220
457000 904220
458000 904220
459000 904220
460000 904220
461000 904220
462000 904220
463000 904220
464000 904220
465000 904220
466000 904220
467000 904220
468000 904220
469000 904220
470000 904220
471000 904220
472000 904220
473000 904220
474000 904220
475000 904220
476000 904220
477000 904220
478000 904220
479000 904220
480000 904220
481000 904220
482000 904220
483000 904220
484000 904220
485000 904220
486000 904220
487000 904220
488000 904220
489000 904220
490000 904220
491000 904220
492000 904220
493000 904220
494000 904220
495000 904220
496000 904220
497000 904220
498000 904220
499000 904220
500000 904220
501000 904220
502000 904220
503000 904220
504000 904220
505000 904220
506000 904220
507000 904220
508000 904220
509000 904220
510000 904220
511000

124000 341008
125000 341008
126000 341008
127000 341008
128000 341008
129000 341008
130000 341008
131000 341008
132000 341008
133000 341008
134000 341008
135000 341008
136000 341008
137000 341008
138000 341008
139000 341008
140000 341008
141000 341008
142000 341008
143000 341008
144000 341008
145000 341008
146000 341008
147000 341008
148000 341008
149000 341008
150000 341008
151000 341008
152000 341008
153000 341008
154000 341008
155000 341008
156000 341008
157000 341008
158000 341008
159000 341008
160000 341008
161000 341008
162000 341008
163000 341008
164000 341008
165000 341008
166000 341008
167000 341008
168000 341008
169000 341008
170000 341008
171000 341008
172000 341008
173000 341008
174000 341008
175000 341008
176000 341008
177000 341008
178000 341008
179000 341008
180000 341008
181000 341008
182000 341008
183000 341008
184000 341008
185000 341008
186000 341008
187000 341008
188000 341008
189000 341008
190000 341008
191000 341008
192000 341008
193000 341008
194000 341008
195000

In [16]:
#plt.hist(noun_sentence_df_T.loc['great fixer']['Listing_Error_List'])

In [17]:
#plt.hist(noun_sentence_df_T.loc['great investor']['Listing_Error_List'])